# Modelo de Volumes de Diesel Vendidos a Usinas Sucroalcooleiras

# Setup Inicial

Carregando linhas com configurações iniciais já escritas em notebook base

No bloco de comando "%run" não pode ter nenhum outra linha de comando ou comentário

In [51]:
%run nb00_Setup_Usinas{}

# Funções

In [52]:
def mape(real, pred):
    '''
        Função que calcula o erro absoluto percentual médio.
    
    Parâmetros
    ----------
        real: pd.DataFrame
            Dataframe com os dados reais
        pred: pd.DataFrame
            Dataframe com os dados previstos
    Result:
    --------
        mape: float
            erro absoluto percentual médio 
    '''

    mape = np.mean(np.abs((pred - real) / real))
    return(mape)

In [53]:
def mape_ano(df):
    '''
        Função que calcula o erro absoluto percentual médio anual
    
    Parâmetros
    ----------
        df: pd.DataFrame
            Dataframe com os dados reais e previstos no periodo
    Result:
    --------
        result: pd.DataFrame
            data frame contendo o erro absoluto percentual médio de cada ano existende em df
    '''
    result = pd.DataFrame()
    for i in df['ds'].dt.year.unique()[:-2]:
        y = df[df['ds'].dt.strftime('%Y')==str(i)]['y']
        yhat = df[df['ds'].dt.strftime('%Y')==str(i)]['yhat']
        result = result.append({'Ano':str(i), 'Mape':mape(y,yhat)}, ignore_index=True)
    return(result)

In [54]:
def ajuste_dados(df, df2, filtro='REGIAO3', agrega=True, regiao='Brasil'): 
    '''
        Auxilia no ajuste dos dados preliminar a previsão dos modelos.

    Parâmetros
    ----------
        df: pd.DataFrame
            Dataframe com os dados de histórico de vendas
        df2: pd.DataFrame
            Dataframe com os dados auxiliares de safra
        filtro: string
            Nome da variável(coluna) que formou o agrupamento regional a ser considerado no processamento
        agrega: Bolean
            Variável que define se os dados auxiliares irão compor o modelo(True) ou não(False)
        regiao: string
            Nome da região a ser considerada no processamento
    Result:
    --------
        hist: pd.DataFrame
           Dataframe com os dados prontos para inserir nos modelos de previsão  
    '''
    try:
        if (regiao!='Brasil'):
            df = df[df[filtro]==regiao]
            df2 = df2[df2[filtro]==regiao]
    except Exception as e:
        return(help(ajuste_dados))

        
    hist = df.groupby(['Data','IMP'])['Vol_Total'].sum().reset_index()
    hist = hist.pivot_table(values='Vol_Total', index='Data', columns='IMP').reset_index()
    hist.rename_axis(None, axis=1, inplace=True)
    hist.columns = ['ds','Vol_Cliente', 'Vol_NaoCliente']
    hist['y'] = hist[['Vol_Cliente', 'Vol_NaoCliente']].sum(axis=1)

    if agrega==True:
        sf = df2.pivot_table(values='sugarcane_crop', index=['data_apuracao'], columns=['product'], aggfunc='sum').reset_index()
        sf = sf.resample('MS',on='data_apuracao').sum().reset_index()
        hist = hist.merge(sf, how='left', left_on='ds', right_on='data_apuracao')
        hist.drop('data_apuracao', axis=1, inplace=True)
        hist.dropna(inplace=True)                                                   # remove os NaN para o modelo poder performar na presença de dados
    return(hist)

In [55]:
def processa_modelo(df_orig, df_modelo):
    '''
        Auxilia no processamento do modelo, por meio da função modelo_final em nb00_Setup_Usinas.
            
    Parâmetros
    ----------
        df_orig: pd.DataFrame
            Dataframe com as componentes do Tamanho de Mercado (Clientes e Não Clientes)
        df_modelo: pd.DataFrame
            Dataframe com os dados a serem ajustados pelos modelos prophet, 
                contendo ou não regressores auxiliares
    Result:
    --------
        pred: pd.DataFrame
            data freme de previsão associados as componentes do Tamanho de Mercado 
                (Clientes e Não Clientes)
        model: Prophet.model
            Modelo prophet ajustado (fitted)
        pri: int
            posição (index) do último dado real, início da previsão 
    '''
    
    pred, model = modelo_final(df_modelo)
    
    pred = pred.merge(df_orig[['ds', 'Vol_Cliente','Vol_NaoCliente']], how='left', on='ds')        # agregando as variáveis originais

    pri = pred.loc[pd.isna(pred['y']), :].index[0]-1                                                # Ultimo dado real = inicio da previsão                                          

    aux = pred.loc[~pred['y'].isna()][['ds', 'y', 'Vol_Cliente','Vol_NaoCliente']][pri-12:]
    aux['Vol_Cli'] = aux['Vol_Cliente']/aux['y']
    aux['Vol_NaoCli'] = aux['Vol_NaoCliente']/aux['y']
    pred['Vol_Cli'] = pred[pri:]['yhat']*aux['Vol_Cli'].values
    pred['Vol_NCli'] = pred[pri:]['yhat']*aux['Vol_NaoCli'].values

    pred['Vol_Cliente'].fillna(pred['Vol_Cli'], inplace=True)
    pred['Vol_NaoCliente'].fillna(pred['Vol_NCli'], inplace=True)
    pred = pred[pred.columns[:-2]]
    
    return(pred, model, pri)

In [56]:
def resultado(regiao='Brasil', filtro='REGIAO3', chave=False):
    '''
        Resultados dos modelos para cada regiao definina no filtro.
            
    Parâmetros
    ----------
        regiao: string
            Nome da região a ser considerada no processamento        
        filtro: string
            Nome da variável(coluna) que formou o agrupamento regional a ser considerado no processamento
        chave: Bolean
            Variável que define se os dados auxiliares irão compor o modelo(True) ou não(False)
    Result:
    --------
        pred: pd.DataFrame
            data freme de previsão associados as componentes do Tamanho de Mercado 
                (Clientes e Não Clientes)
        model: Prophet.model
            Modelo prophet ajustado (fitted)
        pri: int
            posição (index) do último dado real, início da previsão 
        metric: pd.DataFrame
            Dataframe com as métricas de todos os modelos ajustados (metricas_modelos em nb00_Setup_Usinas)
            Métricas retornadas: MSE, MAE, RMSEe MAPE

    '''
    x = ajuste_dados(hist_usinas, safra, filtro=filtro, agrega=chave, regiao=regiao)
    df = x.drop(['Vol_Cliente','Vol_NaoCliente'], axis=1)

    train, test = dividir_train_test(df, num_meses=12)                              # ver função em nb00_Setup_Usinas
    metric = pd.DataFrame(metricas_modelos(df, train, test, order=False))           # ver função em nb00_Setup_Usinas
    metric = pd.concat([metric.drop(['metricas'], axis=1), metric['metricas'].apply(pd.Series)], axis=1)
    metric = metric.sort_values('MAPE').reset_index(drop=True)

    Best_Reg = metric[metric['MAPE'] == metric['MAPE'].min()].iloc[0,:][1]
    lista_best = df.columns[0:2].tolist() # usando a variável de Volume
    lista_best.extend(Best_Reg)

    pred, modelo, pri = processa_modelo(x, df[lista_best])
    return(pred, modelo, pri, metric)

In [57]:
def grafico(df, pri, titulo=''):
    '''
        Grafico das séries de dados dos modelos de previsão de volume geral incluindo clientes e não clientes.

    Parâmetros
    ----------
        df: pd.DataFrame
            Dataframe com os dados a serem plotados
        pri: int
            posição (index) do último dado real, início da previsão
        titulo: string
            título do gráfico
    Result:
    --------
        fig: objeto plotly.graph_objects
            gráfico das séries de dados  
    '''
    cores = ['green','cadetblue','gray','purple', 'gold']

    fig = go.Figure()

    fig.add_trace(go.Scatter(                                   # linha de tendência
        x=df['ds'],
        y=df['trend'],
        line=dict(color='black', width=0.5),
        name='Tendência',
        #legendgroup = 'Modelo'
        #legendgrouptitle_text='Model'      
    ))

    fig.add_trace(go.Scatter(                                   # Intervalo inferio
        x=df['ds'],
        y=df['yhat_lower'],
        marker={'color': 'rgba(0,0,0,0)'},
        showlegend=False,
        hoverinfo='none',
        #legendgroup = 'Modelo' 
    ))

    fig.add_trace(go.Scatter(                                   # intervalo superior - preenchido
        x=df['ds'],
        y=df['yhat_upper'],
        fill='tonexty',
        fillcolor='rgba(255,192,203,0.5)',
        line_color='rgba(255,255,255,0)',
        name='Confidence',
        hoverinfo='none',
        mode='none',
        #legendgroup = 'Modelo' 
    ))

    fig.add_trace(go.Scatter(                                   # Linha do modelo
        x = df['ds'],
        y = df['yhat'],
        line = dict(color='red', width=2),
        name = 'Previsão',
        #legendgroup = 'Modelo' 
    ))
       
    for c,i in enumerate(df.columns[-4:-2]):                      # linhas dos clientes 
        fig.add_trace(go.Scatter(
            name = i,
            x = df['ds'],
            y = df[i],
            line = {'color': cores[c],'dash':'dot'},
            visible=True,
            stackgroup='one',
            #legendgroup = 'Real' 
        ))
        
    fig.add_trace(go.Scatter(                                   # Dados reais
        x=df['ds'],
        y=df['y'],
        mode='lines+markers',
        marker={'color': 'rgb(70,130,180)',
                'size' : 3,
                'line' : {'color': 'rgb(70,130,180)',
                        'width': .75}
                },
        name='Mercado',
        #legendgroup = 'Real'
        #legendgrouptitle_text='Real1'
    ))

    fig.add_annotation(
            showarrow=False,
            text='Gerado em ' + dt.now(tz_SP).strftime("%d %b %Y às %H:%M:%S"),
            font=dict(size=12),
            align="right",
            x=1.0,
            y=-0.5,
            xref='paper',
            yref='paper',
            xanchor='right',
            yanchor='bottom',)

    fig.update_xaxes(matches='x', rangeslider_visible=True, rangeslider_thickness = 0.1)

    # Área de previsão
    fig.add_vrect(x0=df['ds'][pri], x1=df['ds'].max(), 
                annotation_text="Previsão",
                annotation_position="bottom left",  
                annotation_font_size=14,
                annotation_font_color="white",
                #fillcolor="gray",
                opacity=0.15,
                line_width=0)

    fig.update_layout(
        shapes=[
            dict(type="line", xref="x", yref="y",
                    x0=df['ds'][pri], x1=df['ds'][pri], 
                    y0=0, y1=df['y'].max()*1.05,
                    line=dict(color="black",
                            width=2,
                            dash="dot")
                )]
    )

    fig.update_layout(
        template ='plotly_white',
        title = titulo,
        #xaxis_title = "Período",
        yaxis_title = "Volume (m³)",
        #legend_title = "Legenda",
        #legend = dict (itemclick = "toggle"),
        font=dict(family = "Courier New, monospace",
                  size = 14,
                  color = "royalblue"),
        height=600
    )

    return(fig)


# Leitura dos Dados 

In [58]:
hist_usinas = pd.read_parquet(abfss_path_enriched + 'usinas/usi03_vendas_impt_mes.parquet',
                            storage_options = {'linked_service' : linked_service_enriched})

safra = pd.read_parquet(abfss_path_enriched.split('mercado')[0]+'safra_agroconsult/sugarcaneCrop_tratado.parquet',
                            storage_options = {'linked_service' : linked_service_enriched})

# Adequação do Dataframe de Safras

In [59]:

safra['data_apuracao'] = pd.to_datetime(safra['data_apuracao'],                 # String to data ignorando erros
                                        format='%d/%m/%Y',
                                        errors='coerce')
safra['sugarcane_crop'] = safra['sugarcane_crop'].astype(float)                 # String to float

safra = (safra[(safra['source']=='MAPA') &                                      # Filtro informações do MAPA    
               #(safra['product']=='Cana') &                            
               (safra['data_apuracao']>='2016-01-01')]                          # Início da série
            .sort_values('data_apuracao')
            .reset_index(drop=True)
        )

reg = {'Norte'       :'N-NE',                                                   # Dic para auxiliar nas novas regionais
       'Nordeste'    :'N-NE',
       'Sul'         :'CO-S-SE',
       'Centro-Oeste':'CO-S-SE',
       'Sudeste'     :'CO-S-SE'}

safra['REGIAO2'] = [reg[x] if x!='-' else np.nan for x in safra['region']]      # Novas Regiões
safra['REGIAO3'] = np.where(safra['state']=='São Paulo', 'SP', safra['REGIAO2'])


# Correlação entre as Variáveis

In [60]:
regiao = ['Brasil'] + hist_usinas['REGIAO3'].unique().tolist()
# fig, axs = plt.subplots(1, 4, figsize=(25, 5), sharey=True)

for a, i in enumerate(regiao):
    x = ajuste_dados(hist_usinas, safra, filtro='REGIAO3', agrega=True, regiao=i)
    df = x.drop(['Vol_Cliente','Vol_NaoCliente'], axis=1)
    corr, p_values = stats.spearmanr(df[['y','Açúcar','Cana','Etanol Anidro Total','Etanol Hidratado Total']])
    mask = np.invert(np.tril(p_values<0.05)) # plotar somente as correlações significativas
    labels = ['Volume de Diesel','Açúcar','Cana','Etanol Anidro Total','Etanol Hidratado Total']

    # sns.heatmap(abs(corr), annot=True, cmap='Blues',linewidth=1, fmt='.2f',cbar=False,
    #             mask=mask, xticklabels=labels, yticklabels=labels, ax=axs[a])
    # axs[a].set_title(i) 
#fig.show()
pd.DataFrame(corr).to_csv(abfss_path_enriched + 'usinas/resultados/to_del.csv', 
                     storage_options = {'linked_service':linked_service_enriched})

# Resultados

In [61]:
# montar ambiente para salvar resultados
try:
    mount_dir = '/mount'

    mssparkutils.fs.mount( 
        abfss_path_enriched + 'usinas/resultados', 
        mount_dir, 
        {"linkedService":linked_service_enriched}
    )

    job = mssparkutils.env.getJobId()
    dir_m = '/synfs/{}{}/'.format(job, mount_dir)
except:
    print('Falha da montagen do diretório.\nMétricas e Graficos não serão salvos')

In [62]:
for i in regiao:
    # fit dos modelos
    pred1, modelo1, pri1, metric1 = resultado(regiao=i, filtro='REGIAO3', chave=False)
    pred2, modelo2, pri2, metric2 = resultado(regiao=i, filtro='REGIAO3', chave=True)

    # métrica anual
    res_mape = mape_ano(pred1).merge(mape_ano(pred2), how='left', on='Ano')
    res_mape.columns = ['ANO', 'MAPE_Base-Model', 'MAPE_Best-Model']

    # Salvando previsões
    arquivo = i + '_' + dt.now(tz_SP).strftime('%Y%m%d')
    pred2['data_str'] = pred2['ds'].dt.strftime('%Y-%m-%d').astype('string')
    pred2['regiao'] = i                                                                         # Auxilia no join e filtro da visualização
    
    pred2.to_parquet(abfss_path_enriched + 'usinas/usi05_Previsao_'+ i +'.parquet',             # modelo geral
                    storage_options = {'linked_service':linked_service_enriched})
    
    pred2.to_parquet(abfss_path_enriched + 'usinas/resultados/Previsao_'+ arquivo +'.parquet', 
                     storage_options = {'linked_service':linked_service_enriched})
    
    #Salvando Métricas
    pd.set_option('display.expand_frame_repr', False)        # Forçar que não exista quebra de páginas no print do dataframe

    with open(dir_m + 'Metricas_' + arquivo + '.txt', 'w') as f:
        print('RESULTADOS DO MODELO\n', file=f)
        print('Processado em: ' + dt.now(tz_SP).strftime('%d/%m/%Y às %H:%M:%S') + '\n', file=f)
        print('Agrupemento Geográfico: ' + i + '\n', file=f)
        print('Regressores do melhor modelo:', file=f)
        print(list(modelo2.extra_regressors.keys()), file=f)
        print('\n', file=f)
        print('MAPE anual:', file=f)
        print(res_mape, file=f)
        print('\n', file=f)
        print('Metricas dos modelos processados\n', file=f)
        print(metric2, file=f)

    # Salvando grafico
    fig = grafico(pred2, pri2, titulo=(
                        'Performance do  Modelo de Previsão para Volume de Diesel<br><sup>'+
                        'Usinas Sucroalcooleiras ('+ i + ')</sup>'))
    fig = plotly.offline.plot(fig, filename = dir_m + 'Grafico_'+arquivo +'.html', auto_open=False)

In [63]:
mssparkutils.fs.unmount(mount_dir) 